<a href="https://colab.research.google.com/github/slnarayan/AddressEvaluationLLM/blob/main/AddressEvaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install python-dotenv
!pip install openai
#!pip install cohere
#!pip install tiktoken

In [23]:
import os
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

llm_model = "gpt-3.5-turbo-0301"

def get_completion(prompt, model=llm_model):
  client = OpenAI( api_key = os.environ['OPENAI_API_KEY'],)
  messages = [{"role": "user", "content":prompt}]
  response = client.chat.completions.create(
      model=model,
      messages=messages,
      temperature=0.0,
  )
  return response.choices[0].message.content

In [24]:
get_completion("What is 1+1?")

'As an AI language model, I can tell you that the answer to 1+1 is 2.'

In [25]:
!pip install --upgrade langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.3/803.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.1 MB/s eta 0:00:00


In [93]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
import pandas as pd
import numpy as np

chat = ChatOpenAI(temperature=0.0, model=llm_model)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [204]:
file = 'address.csv'
#loader = CSVLoader(file_path=file)
#data = loader.load()
df = pd.read_csv('address.csv')
df = df.replace(np.nan, '')
df['status']= ''

def get_addresses(df):
  #Format the original address
  l1 = df['line1']
  l2 = df['line2']
  city = df['city']
  state = df['state']
  zip = str(df['zip'])

  #Format the standardized address
  s_l1 = df['line1_s']
  s_l2 = df['line2_s']
  s_city = df['city_s']
  s_state = df['state_s']
  s_zip = str(df['zip_s'])

  org_addr = 'Valid Address: '+l1+', '+l2+', '+city+', '+state+', '+zip
  std_addr = 'Standardized Address: '+s_l1+', '+s_l2+', '+s_city+', '+s_state+', '+s_zip
  return org_addr, std_addr

'''
def get_addresses_DNU(df):
  #Format the original address
  l1 = df.loc[:0]['line1'].tolist()
  l2 = df.loc[:0]['line2'].tolist()
  city = df.loc[:0]['city'].tolist()
  state = df.loc[:0]['state'].tolist()
  zip = df.loc[:0]['zip'].astype(str).tolist()

  #Format the standardized address
  s_l1 = df.loc[:0]['line1_s'].tolist()
  s_l2 = df.loc[:0]['line2_s'].tolist()
  s_city = df.loc[:0]['city_s'].tolist()
  s_state = df.loc[:0]['state_s'].tolist()
  s_zip = df.loc[:0]['zip_s'].astype(str).tolist()

  org_addr = 'Valid Address: '+l1[0]+', '+l2[0]+', '+city[0]+', '+state[0]+', '+zip[0]
  std_addr = 'Standardized Address: '+s_l1[0]+', '+s_l2[0]+', '+s_city[0]+', '+s_state[0]+', '+s_zip[0]
  return org_addr, std_addr
  '''

"\ndef get_addresses_DNU(df):\n  #Format the original address\n  l1 = df.loc[:0]['line1'].tolist()\n  l2 = df.loc[:0]['line2'].tolist()\n  city = df.loc[:0]['city'].tolist()\n  state = df.loc[:0]['state'].tolist()\n  zip = df.loc[:0]['zip'].astype(str).tolist()\n\n  #Format the standardized address\n  s_l1 = df.loc[:0]['line1_s'].tolist()\n  s_l2 = df.loc[:0]['line2_s'].tolist()\n  s_city = df.loc[:0]['city_s'].tolist()\n  s_state = df.loc[:0]['state_s'].tolist()\n  s_zip = df.loc[:0]['zip_s'].astype(str).tolist()\n\n  org_addr = 'Valid Address: '+l1[0]+', '+l2[0]+', '+city[0]+', '+state[0]+', '+zip[0]\n  std_addr = 'Standardized Address: '+s_l1[0]+', '+s_l2[0]+', '+s_city[0]+', '+s_state[0]+', '+s_zip[0]\n  return org_addr, std_addr\n  "

In [205]:
template_string = """Compare the two address that is given in text \
delimited by triple backticks and return \
'yes' if format matches otherwise return 'no'.\
text: ```{org}\n
{std}```
"""

In [206]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string)

In [209]:
for index, row in df.iterrows():
  org_addr, std_addr = get_addresses(row)
  prompt_messages = prompt_template.format_messages(
    org = org_addr,
    std = std_addr
  )
  print(prompt_messages[0])
  chat_response = chat(prompt_messages)
  df['status'][index]= chat_response.content

content="Compare the two address that is given in text delimited by triple backticks and return 'yes' if format matches otherwise return 'no'.text: ```Valid Address: 10 Florence st, #201, malden, MA, 2148.0\n\nStandardized Address: 1018 Oleander Ln, , Waxhaw, NC, ```\n"


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


content="Compare the two address that is given in text delimited by triple backticks and return 'yes' if format matches otherwise return 'no'.text: ```Valid Address: 1018 Oleander Ln, , Waxhaw, NC, \n\nStandardized Address: 1018 Oleander Ln, , Waxhaw, NC, 28173.0```\n"


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [210]:
print(df)

              line1 line2    city state     zip           line1_s line2_s  \
0    10 Florence st  #201  malden    MA  2148.0  1018 Oleander Ln           
1  1018 Oleander Ln        Waxhaw    NC          1018 Oleander Ln           

   city_s state_s    zip_s status  
0  Waxhaw      NC              no  
1  Waxhaw      NC  28173.0    yes  


no
